In [2]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium
import plotly.graph_objs as go
import random

In [3]:
mrt_df = pd.read_csv('../../data/Isochrone_data/mrt_station_colours.csv')
#color remapping
new_color_map = {
    'red': '#cb2721',
    'yellow': '#e69c3d',
    'green': '#159048',
    'blue': '#0b569d',
    'purple': '#7d4687',
    'brown': '#a67440',
    'grey': '#7c8981',
}
mrt_df['Color'] = mrt_df['Color'].map(new_color_map)
dict_mrt_color = dict(zip(mrt_df['MRT.Name'], mrt_df['Color']))
bicycle_isochrones = pd.read_json('../../data/Isochrone_data/bicycle_isochrones.json', orient='records', lines=True)
bus_isochrones = pd.read_json('../../data/Isochrone_data/bus_isochrones.json', orient='records', lines=True)
mrt_isochrones = pd.read_json('../../data/Isochrone_data/mrt_isochrones.json', orient='records', lines=True)
public_isochrones = pd.read_json('../../data/Isochrone_data/public_isochrones.json', orient='records', lines=True)

def get_isochrone(name, mode, cutoff):
    # Construct the variable name dynamically
    variable_name = f"{mode}_isochrones"    
    df = globals()[variable_name] 
    filtered_df = df[(df['MRT.Name'] == name)]
    isochrone = filtered_df[f'isochrone_{cutoff}M'].iloc[0]
    return isochrone

Note: coords change slightly, identify by MRT.Name instead.

In [4]:
lat,lon = mrt_df[mrt_df['MRT.Name'] == 'BISHAN MRT STATION'][['Latitude', 'Longitude']].values[0]
print(lat, lon)
print(bicycle_isochrones[bicycle_isochrones['MRT.Name'] == 'BISHAN MRT STATION'][['Latitude', 'Longitude']].values[0])
#get_isochrone(name='BISHAN MRT STATION', mode="bicycle", cutoff=30)

1.351315801 103.8491403
[  1.3513158 103.8491403]


Visualise all isochrones

In [19]:
import plotly.graph_objs as go
import random

f = go.FigureWidget()
f.layout.hovermode = 'closest'
f.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = 5
random.seed = 42
for i in range(num_of_traces):
    y = [random.random() + i / 2 for _ in range(100)]
    trace = go.Scatter(y=y, mode='lines', line={ 'width': default_linewidth })
    f.add_trace(trace)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
    for i,_ in enumerate(f.data):
        f.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
    print(points.trace_index)

# we need to add the on_click event to each trace separately       
for i in range(len(f.data) ):
    f.data[i].on_click(update_trace)

# let's show the figure 
f

FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '5ac5ed48-3c25-426d-b0d4-d353f81a509f',
              'y': [0.4962154802629043, 0.9011995150503584, 0.9945991091536313,
                    0.8325024402851625, 0.36050547070865435, 0.9308989204080149,
                    0.1377259536583405, 0.8134346919464712, 0.4744761709609766,
                    0.7418107284358176, 0.38130474595043007, 0.8108907699657129,
                    0.5184397805519372, 0.6750955399090751, 0.3011963645167035,
                    0.10358661899416921, 0.1570989860865476, 0.05104649439382114,
                    0.6120165954502435, 0.22055046935353007, 0.6305555169462516,
                    0.7773620186717793, 0.539455222035242, 0.5154591851336301,
                    0.8875193537253647, 0.31505960125193844, 0.9870763830199419,
                    0.055980605524035054, 0.8743220464995507, 0.9044119277853698,
                 

In [3]:
import plotly.graph_objects as go


fig = go.Figure(go.Scattermapbox(lat=[36,37,38],
                                 lon=[26,27,28],
                                 mode='markers',
                                 marker={'size':9,'color':'purple'},
                                 selected=go.scattermapbox.Selected(marker = {"color":"red", "size":25})
                                 )
                )

fig.update_layout(clickmode='event+select',
                  mapbox = {'style': "stamen-terrain",
                            'center': {'lon': 27.0, 'lat': 37.0 },
                            'zoom': 6},
                  margin = dict(l=0, r=0, t=0, b=0)
                  )

fig.show()

In [26]:
from plotly.subplots import make_subplots
f = go.FigureWidget(make_subplots(rows=2, cols=1))

trace1 = go.Scatter(x=[3, 4, 5], y=[10, 11, 12], name='trae1')
trace2 = go.Scatter(x=[6, 7, 8], y=[13, 14, 15])

f.add_trace(trace1, row=1, col=1)
f.add_trace(trace2, row=2, col=1)

def print_coordinates(trace, points, selector):
    if points.point_inds:  # Ensure a point is clicked
        # Extract the coordinates of the clicked marker
        print(trace)
        clicked_lon = trace.x[1]
        clicked_lat = trace.y[1]
        # Print the coordinates
        print(f"Clicked marker coordinates: Lat {clicked_lat}, Lon {clicked_lon}")

for trace in f.data:
    trace.on_click(print_coordinates)


a = 3
minutes = '%02d' % (3)
seconds = '%02d' % (4)
cutoff = minutes+"M"+seconds+"S"
print(cutoff)

03M04S


Plotting of the Graph of MRT Stations (Can click and toggle the size of the MRT Stations)

In [82]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()
fig.layout.hovermode = 'closest'
default_linewidth = 2
highlighted_linewidth = 3

# Fetch and add isochrones for each MRT station
for _, row in mrt_df.iterrows():
    mrt_name = row['MRT.Name']
    # Add isochrone as a trace
    color = row['Color']
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[row['Longitude']],
        lat=[row['Latitude']],
        text = [mrt_name],
        marker=dict(size=10,color=color),
        selected=go.scattermapbox.Selected(marker = {"size":25}),
        unselected=go.scattermapbox.Unselected(marker = {"color":"grey", "size":5})
    ))
"""def click_handler(trace, points, selector):
    lat = trace['lat'][0]
    lon = trace['lon'][0]
    isochrone_geojson = fetch_isochrone(lat, lon, modes="TRANSIT")
    geometry = isochrone_geojson['geometry']
    polygon_color = mrt_color[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['Color'].iloc[0]
    if 'coordinates' in geometry:
        coordinates = geometry['coordinates'][0][0]
        # Assuming coordinates are in [lon, lat] format
        lon_coords = [coord[0] for coord in coordinates]
        lat_coords = [coord[1] for coord in coordinates]
        fig.add_trace(go.Scattermapbox(
                    mode="lines",
                    lon=lon_coords,
                    lat=lat_coords,
                    name=f"Isochrone {lat}, {lon}",
                    line=dict(width=1, color=polygon_color),
                    fill = "toself",
                    marker=dict(size=0),
                    text=[mrt_name],  # Set text for hover
                    hoverinfo='text'
                ))
    
    print("Hi")
for trace in fig.data:
    trace.on_click(click_handler)"""
# Define layout for the map
fig.update_layout(
    clickmode='event+select',
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)

fig

In [23]:

from dash import dcc
from dash import Dash, html, dcc
import json
app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(['Cycling', 'MRT', 'Bus', 'Cycling + Bus', 'Cycling + MRT'], 'Cycling'),
    dcc.Checklist(
        [
        {
            "label": html.Div(['Cycling'], style={'color': 'Gold', 'font-size': 20}),
            "value": "Cycling",
        },
        {
            "label": html.Div(['MRT'], style={'color': 'Red', 'font-size': 20}),
            "value": "MRT",
        },
        {
            "label": html.Div(['Bus'], style={'color': 'LightGreen', 'font-size': 20}),
            "value": "Bus",
        },
    ], value=['Cycling'],
    labelStyle={"display": "flex", "align-items": "center"},
    ),
    dcc.Slider(0, 20, 1, value=18),
    dcc.Graph(
        id='map',
        figure=fig 
    )
])

if __name__ == '__main__':
    app.run(debug=True)

In [28]:
from dash_extensions.javascript import assign
from dash import Dash
from dash_leaflet import Map, TileLayer

# Define JavaScript function to print coordinates.
js_function = """
function(e, ctx) {
    console.log(`You clicked at ${e.latlng.lat}, ${e.latlng.lng}.`);
}
"""

# Assign JavaScript function to the click event handler.
event_handlers = dict(click=assign(js_function))

# Create the Dash app.
app = Dash(__name__)
app.layout = Map(children=[TileLayer()], eventHandlers=event_handlers,
                 style={'height': '50vh'}, center=[56, 10], zoom=6)

if __name__ == '__main__':
    app.run_server()

In [80]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Fetch and add isochrones for each MRT station
for _, row in mrt_df.iterrows():
    mrt_name = row['MRT.Name']
    color = row['Color']    
    isochrone = get_isochrone(name=mrt_name, mode="bicycle", cutoff=15)

    lon_coords = [coord[0] for coord in isochrone]
    lat_coords = [coord[1] for coord in isochrone]
    
    # Add isochrone as a trace
    fig.add_trace(go.Scattermapbox(
        mode="lines",
        lon=lon_coords,
        lat=lat_coords,
        name=f"Isochrone {lat}, {lon}",
        line=dict(width=1, color=color),
        fill="toself",
        marker=dict(size=0),
        text=[mrt_name],  # Set text for hover
        hoverinfo='text'
    ))
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[lon],
        lat=[lat],
        text=[mrt_name],
        name=mrt_name,
        marker=dict(size=10, color=color),
        textposition="bottom center",
    ))

# Define layout for the map
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend=False
)

# Show the map
fig.show()
